In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import rasterio
import joblib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from torchvision import models

# ----------------------
# Device & CSV
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

df = pd.read_csv("pseudo_labels_with_exg.csv")
file_names = df["image"].tolist()
labels = df["pseudo_label"].tolist()
base_dir = r"E:\Early Crop Stress Prediction Before Visible Damage\Dataset"

# ----------------------
# Helper function: read tif
# ----------------------
def read_tif(path):
    with rasterio.open(path) as src:
        img = src.read(1)
    img = cv2.resize(img, (128, 128)).astype(np.float32)
    img = (img - img.min()) / (img.max() - img.min() + 1e-6)
    return img

# ----------------------
# Gradient Boosting features
# ----------------------
features = []
for _, row in df.iterrows():
    rgb = cv2.imread(os.path.join(base_dir, "RGB", row["image"]))
    rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
    rgb = cv2.resize(rgb, (128,128)).astype(np.float32)/255.0

    R, G, B = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    exg = 2*G - R - B

    ndvi = read_tif(os.path.join(base_dir, "NDVI", row["image"]))
    vh   = read_tif(os.path.join(base_dir, "SAR","VH",row["image"]))
    vv   = read_tif(os.path.join(base_dir, "SAR","VV",row["image"]))

    features.append([ndvi.mean(), vh.mean(), vv.mean(), exg.mean()])

X = np.array(features)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ----------------------
# Gradient Boosting
# ----------------------
gb = GradientBoostingClassifier(n_estimators=150, learning_rate=0.05, max_depth=3, random_state=42)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)

print("=== GRADIENT BOOSTING REPORT ===")
print(classification_report(y_test, gb_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, gb_pred))

joblib.dump(gb, "gb_crop_stress_model1.pkl")
print("Gradient Boosting model saved")

print("Gradient Boosting Accuracy")
print("Train:", accuracy_score(y_train, gb.predict(X_train)))
print("Test :", accuracy_score(y_test, gb.predict(X_test)))

# ----------------------
# Data Augmentation for CNN
# ----------------------
def augment_image(img):
    """Augment 6-channel image (RGB+NDVI+VH+VV)"""
    img = img.astype(np.float32)
    H, W, C = img.shape

    # Horizontal flip
    if random.random() > 0.5:
        img = np.flip(img, axis=1).copy()  # <-- add .copy()
    # Vertical flip
    if random.random() > 0.5:
        img = np.flip(img, axis=0).copy()  # <-- add .copy()
    # Rotation -15 to 15 degrees
    angle = random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((W/2, H/2), angle, 1)
    for c in range(C):
        img[:,:,c] = cv2.warpAffine(img[:,:,c], M, (W,H), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    # Random brightness on RGB only
    if random.random() > 0.5:
        factor = random.uniform(0.8, 1.2)
        img[:,:,:3] = np.clip(img[:,:,:3] * factor, 0, 1)
    return img


# ----------------------
# Dataset Class
# ----------------------
class CropDataset(Dataset):
    def __init__(self, base_dir, files, labels, augment=False):
        self.base_dir = base_dir
        self.files = files
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        name = self.files[idx]
        label = self.labels[idx]

        rgb = cv2.imread(os.path.join(self.base_dir,"RGB",name))
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        rgb = cv2.resize(rgb,(128,128)).astype(np.float32)/255.0

        ndvi = read_tif(os.path.join(self.base_dir,"NDVI",name))[...,None]
        vh   = read_tif(os.path.join(self.base_dir,"SAR","VH",name))[...,None]
        vv   = read_tif(os.path.join(self.base_dir,"SAR","VV",name))[...,None]

        img = np.concatenate([rgb, ndvi, vh, vv], axis=2)

        if self.augment:
            img = augment_image(img)

        img = torch.tensor(img).permute(2,0,1)
        return img, torch.tensor(label)

# ----------------------
# Prepare DataLoaders
# ----------------------
dataset = CropDataset(base_dir, file_names, labels, augment=True)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)

# ----------------------
# CNN Model
# ----------------------
class ResNetCropStress(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(weights=None)
        self.model.conv1 = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)
    def forward(self, x):
        return self.model(x)

model = ResNetCropStress().to(device)
print("Model created successfully")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)

# ----------------------
# CNN Training Loop
# ----------------------
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

# ----------------------
# CNN Evaluation
# ----------------------
model.eval()
preds, true = [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds.extend(torch.argmax(outputs,1).cpu().numpy())
        true.extend(labels.numpy())

print("=== CNN REPORT ===")
print(classification_report(true, preds))
print("Confusion Matrix:\n", confusion_matrix(true, preds))

torch.save(model.state_dict(), "cnn_crop_stress_model1.pth")
print("CNN model saved")


Using device: cpu


c:\Users\vishm\AppData\Local\Programs\Python\Python310\lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


=== GRADIENT BOOSTING REPORT ===
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       133
           1       1.00      1.00      1.00       307

    accuracy                           1.00       440
   macro avg       1.00      1.00      1.00       440
weighted avg       1.00      1.00      1.00       440

Confusion Matrix:
 [[132   1]
 [  0 307]]
Gradient Boosting model saved
Gradient Boosting Accuracy
Train: 1.0
Test : 0.9977272727272727
Model created successfully


c:\Users\vishm\AppData\Local\Programs\Python\Python310\lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Epoch 1/10 - Loss: 0.2919
Epoch 2/10 - Loss: 0.1746
Epoch 3/10 - Loss: 0.1962
Epoch 4/10 - Loss: 0.1969
Epoch 5/10 - Loss: 0.2220
Epoch 6/10 - Loss: 0.2133
Epoch 7/10 - Loss: 0.1922
Epoch 8/10 - Loss: 0.1668
Epoch 9/10 - Loss: 0.2016
Epoch 10/10 - Loss: 0.1715
=== CNN REPORT ===
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       140
           1       0.97      0.97      0.97       300

    accuracy                           0.96       440
   macro avg       0.95      0.95      0.95       440
weighted avg       0.96      0.96      0.96       440

Confusion Matrix:
 [[131   9]
 [ 10 290]]
CNN model saved
